In [49]:
import numpy as np
import pandas as pd
import plotly
from plotly.offline import plot

In [11]:
nodes_df = pd.read_csv("../analysis_nci/node_num_dict.csv")
gene_edges_df = pd.read_csv("../analysis_nci/edgefilter_gene_bind_weight_edge.csv")
drug_edges_df = pd.read_csv("../data-nci/filtered_data/final_drugbank_num.csv")

In [12]:
nodes_df.head()

,node_num,node_name,node_type
0,1,ABL1,gene
1,2,ABL2,gene
2,3,ACAA1,gene
3,4,ACACA,gene
4,5,ACACB,gene


In [13]:
gene_edges_df.head()

,src,dest,conv_bind_weight
0,20,1160,0.203711
1,20,1162,0.208021
2,20,1163,0.382297
3,20,1165,0.204268
4,46,1519,0.235613


In [14]:
drug_edges_df.head()

,Drug,Target
0,2017,1366
1,2017,1556
2,2018,1676
3,2018,1677
4,2019,1


Define Nodepositions for (hopefully) better vizualisations

In [52]:
# Load your data
df1 = pd.read_csv("../data-nci/filtered_data/final_rna.csv")  # Gene x-positions (2016 entries)
df2 = pd.read_csv("../data-nci/filtered_data/final_cnv.csv")  # Gene y-positions (2016 entries)
drug_dict = pd.read_csv("../data-nci/filtered_data/drug_num_dict.csv")  # Drug mappings

# Calculate gene positions
gene_pos_x = df1.iloc[:, 1:].mean(axis=1).values  # Shape (2016,)
gene_pos_y = df2.iloc[:, 1:].mean(axis=1).values  # Shape (2016,)

# Create drug positions
num_drugs = len(drug_dict)
drug_pos_x = np.linspace(gene_pos_x.min(), gene_pos_x.max(), num_drugs)  # Spread drugs along x-range
drug_pos_y = np.full(num_drugs, -2.0)  # All drugs at y=-2 (changed from zeros)

# Combine gene and drug positions
combined_pos_x = np.concatenate([gene_pos_x, drug_pos_x])
combined_pos_y = np.concatenate([gene_pos_y, drug_pos_y])

# ===== NORMALIZATION =====
def normalize(arr):
    """Scale values to range [0, 10] with padding"""
    min_val, max_val = arr.min(), arr.max()
    range_val = max_val - min_val
    if range_val > 0:
        return 10 * (arr - min_val) / range_val
    else:
        return np.full_like(arr, 5)  # All equal -> center at 5

norm_node_x = normalize(combined_pos_x)  # Now in [0, 10]
norm_node_y = normalize(combined_pos_y)  # Now in [0, 10]

norm_node_x = np.abs(norm_node_x) + 0.2

print(f"Final shapes - X: {combined_pos_x.shape}, Y: {combined_pos_y.shape}")

Final shapes - X: (2034,), Y: (2034,)


In [53]:
import plotly.graph_objects as go

# Initialize the figure
fig = go.Figure()

# (1) Add Nodes
node_colors = {
    "drug": "red",
    "gene": "blue"
}

# Assign positions (optional: use layout algorithms like `nx.spring_layout` if needed)
perturbation = 0.1  # Adjust as needed
np.random.seed(42)    # For reproducibility

node_positions = {
    row["node_num"]: (
        norm_node_x[i] + np.random.uniform(-perturbation, perturbation),
        norm_node_y[i] + np.random.uniform(-perturbation, perturbation)
    )
    for i, (_, row) in enumerate(nodes_df.iterrows())
}

# Add nodes to the plot
for _, row in nodes_df.iterrows():
    fig.add_trace(go.Scatter(
        x=[node_positions[row["node_num"]][0]],
        y=[node_positions[row["node_num"]][1]],
        mode="markers+text",
        marker=dict(
            size=20,
            color=node_colors[row["node_type"]],
            line=dict(width=2, color="DarkSlateGrey")
        ),
        text=row["node_name"],
        textposition="top center",
        name=row["node_type"],
        hoverinfo="text",
        hovertext=f"Name: {row['node_name']}<br>Type: {row['node_type']}"
    ))

# (2) Add Gene-Gene Edges (with weights)
for _, row in gene_edges_df.iterrows():
    src_pos = node_positions[row["src"]]
    dest_pos = node_positions[row["dest"]]
    fig.add_trace(go.Scatter(
        x=[src_pos[0], dest_pos[0], None],
        y=[src_pos[1], dest_pos[1], None],
        mode="lines",
        line=dict(width=2 * row["conv_bind_weight"], color="gray"),
        hoverinfo="text",
        hovertext=f"Weight: {row['conv_bind_weight']}",
        showlegend=False
    ))

# (3) Add Drug-Gene Edges (no weights)
for _, row in drug_edges_df.iterrows():
    src_pos = node_positions[row["Drug"]]
    dest_pos = node_positions[row["Target"]]
    fig.add_trace(go.Scatter(
        x=[src_pos[0], dest_pos[0], None],
        y=[src_pos[1], dest_pos[1], None],
        mode="lines",
        line=dict(width=1, color="black", dash="dot"),
        hoverinfo="text",
        hovertext="Drug-Target",
        showlegend=False
    ))

# Customize layout
fig.update_layout(
    title="Drug-Gene Interaction Network",
    showlegend=True,
    hovermode="closest",
    margin=dict(b=20, l=20, r=20, t=40),
    xaxis=dict(type="log", showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    plot_bgcolor="white"
)

fig.show()
plot(fig, filename='network_graph.html', auto_open=True)

'network_graph.html'